# Yelp Data Challenge - Restaurant Recommender

BitTiger DS501

Jun 2017

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline
plt.style.use("ggplot")

In [2]:
df = pd.read_csv('data/last_2_years_restaurant_reviews.csv')

In [3]:
df.head()

,business_id,name,categories,avg_stars,cool,date,funny,review_id,stars,text,type,useful,user_id,count
0,--9e1ONYQuAa-CB_Rrw7Tw,Delmonico Steakhouse,"[Steakhouses, Restaurants, Cajun/Creole]",4.0,0,2015-06-26,0,nCqdz-NW64KazpxqnDr0sQ,1,I mainly went for the ceasar salad prepared ta...,review,0,0XVzm4kVIAaH4eQAxWbhvw,318
1,--9e1ONYQuAa-CB_Rrw7Tw,Delmonico Steakhouse,"[Steakhouses, Restaurants, Cajun/Creole]",4.0,0,2015-06-29,0,iwx6s6yQxc7yjS7NFANZig,4,Nice atmosphere and wonderful service. I had t...,review,0,2aeNFntqY2QDZLADNo8iQQ,318
2,--9e1ONYQuAa-CB_Rrw7Tw,Delmonico Steakhouse,"[Steakhouses, Restaurants, Cajun/Creole]",4.0,0,2015-04-05,0,2HrBENXZTiitcCJfzkELgA,2,To be honest it really quit aweful. First the ...,review,0,WFhv5pMJRDPWSyLnKiWFXA,318
3,--9e1ONYQuAa-CB_Rrw7Tw,Delmonico Steakhouse,"[Steakhouses, Restaurants, Cajun/Creole]",4.0,0,2016-02-16,0,6YNPXoq41qTMZ2TEi0BYUA,2,"The food was decent, but the service was defin...",review,0,2S6gWE-K3DHNcKYYSgN7xA,318
4,--9e1ONYQuAa-CB_Rrw7Tw,Delmonico Steakhouse,"[Steakhouses, Restaurants, Cajun/Creole]",4.0,0,2016-02-08,1,4bQrVUiRZ642odcKCS0OhQ,2,If you're looking for craptastic service and m...,review,1,rCTVWx_Tws2jWi-K89iEyw,318


## 1. Clean data and get rating data 

#### Select relevant columns in the original dataframe

In [99]:
# Get business_id, user_id, stars for recommender

df_select = df[['business_id', 'user_id', 'stars']]
df_select.head(2)

,business_id,user_id,stars
0,--9e1ONYQuAa-CB_Rrw7Tw,0XVzm4kVIAaH4eQAxWbhvw,1
1,--9e1ONYQuAa-CB_Rrw7Tw,2aeNFntqY2QDZLADNo8iQQ,4


#### There are many users that haven't given many reviews, exclude these users from the item-item similarity recommender

**Q**: How do we recommend to these users anyways?

In [100]:
# To be implemented
Review_Num_Limit = 3
df_user_count = df_select.copy()
df_user_count['count'] = 1
df_user_count.head(2)
df_user_count = df_user_count.groupby('user_id')[['count']].sum()
df_user_count = df_user_count[df_user_count['count'] > Review_Num_Limit]
df_merge = pd.merge(df_select, df_user_count, how = 'inner', left_on='user_id', right_index=True)
df_merge = df_merge.reset_index().drop('index', axis = 1)
df_merge.head(2)

,business_id,user_id,stars,count
0,--9e1ONYQuAa-CB_Rrw7Tw,0XVzm4kVIAaH4eQAxWbhvw,1,16
1,2iTsRqUsPGRH1li1WVRvKQ,0XVzm4kVIAaH4eQAxWbhvw,4,16


#### Create utility matrix from records

In [101]:
df_data = df_merge.drop('count', axis = 1)
df_data.head(2)

,business_id,user_id,stars
0,--9e1ONYQuAa-CB_Rrw7Tw,0XVzm4kVIAaH4eQAxWbhvw,1
1,2iTsRqUsPGRH1li1WVRvKQ,0XVzm4kVIAaH4eQAxWbhvw,4


In [102]:
# reconstruct business id
unique_business_id = df_data['business_id'].unique()
business_shape = unique_business_id.shape
print('Number of Unique Business ID: %d' % business_shape[0])
business_df = pd.DataFrame({
        'business_id' :unique_business_id,
        'business_index': xrange(business_shape[0])
    })
business_df.head(2)

Number of Unique Business ID: 4138


,business_id,business_index
0,--9e1ONYQuAa-CB_Rrw7Tw,0
1,2iTsRqUsPGRH1li1WVRvKQ,1


In [103]:
# reconstruct user id
unique_users_id = df_data['user_id'].unique()
user_shape = unique_users_id.shape
print('Number of Unique User ID: %d' % user_shape[0])
user_df = pd.DataFrame({
        'user_id': unique_users_id,
        'user_index': xrange(user_shape[0])
    })
user_df.head(2)

Number of Unique User ID: 18901


,user_id,user_index
0,0XVzm4kVIAaH4eQAxWbhvw,0
1,rCTVWx_Tws2jWi-K89iEyw,1


In [104]:
# inner join the business_df and df_data
df_data = pd.merge(df_data, business_df, how = 'inner', left_on='business_id', right_on = 'business_id')
df_data.head(2)

,business_id,user_id,stars,business_index
0,--9e1ONYQuAa-CB_Rrw7Tw,0XVzm4kVIAaH4eQAxWbhvw,1,0
1,--9e1ONYQuAa-CB_Rrw7Tw,rCTVWx_Tws2jWi-K89iEyw,2,0


In [105]:
df_data = pd.merge(df_data, user_df, how = 'inner', left_on='user_id', right_on = 'user_id')
df_data.head(2)

,business_id,user_id,stars,business_index,user_index
0,--9e1ONYQuAa-CB_Rrw7Tw,0XVzm4kVIAaH4eQAxWbhvw,1,0,0
1,2iTsRqUsPGRH1li1WVRvKQ,0XVzm4kVIAaH4eQAxWbhvw,4,1,0


In [118]:
# Utility matrix can be used with sparse matrix
from scipy import sparse
from sklearn.metrics.pairwise import cosine_similarity
import random
from time import time

In [107]:
# construct sparse matrix
highest_user_id = unique_users_id.shape[0]
highest_business_id = unique_business_id.shape[0]
ratings_mat = sparse.lil_matrix((highest_user_id, highest_business_id))
ratings_mat

<18901x4138 sparse matrix of type '<type 'numpy.float64'>'
	with 0 stored elements in LInked List format>

In [108]:
for _, row in df_data.iterrows():
    # subtract 1 from id's due to match 0 indexing
    ratings_mat[row.user_index, row.business_index] = row.stars

## 2. Item-Item similarity recommender

### Let's reuse the ItemItemRecommender class derived from previous exercise

Hint: we need to make modification to accommodate the dense numpy array

In [126]:
class ItemItemRecommender(object):

    def __init__(self, neighborhood_size):
        self.neighborhood_size = neighborhood_size

    def fit(self, ratings_mat):
        self.ratings_mat = ratings_mat
        self.n_users = ratings_mat.shape[0]
        self.n_items = ratings_mat.shape[1]
        self.item_sim_mat = cosine_similarity(self.ratings_mat.T)
        self._set_neighborhoods()

    def _set_neighborhoods(self):
        least_to_most_sim_indexes = np.argsort(self.item_sim_mat, 1)
        self.neighborhoods = least_to_most_sim_indexes[:, -self.neighborhood_size:]

    def pred_one_user(self, user_id, report_run_time=False):
        start_time = time()
        items_rated_by_this_user = self.ratings_mat[user_id].nonzero()[1]
        # Just initializing so we have somewhere to put rating preds
        out = np.zeros(self.n_items)
        for item_to_rate in range(self.n_items):
            relevant_items = np.intersect1d(self.neighborhoods[item_to_rate],
                                            items_rated_by_this_user,
                                            assume_unique=True)  # assume_unique speeds up intersection op
            out[item_to_rate] = self.ratings_mat[user_id, relevant_items] * \
                self.item_sim_mat[item_to_rate, relevant_items] / \
                self.item_sim_mat[item_to_rate, relevant_items].sum()
        if report_run_time:
            print("Execution time: %f seconds" % (time()-start_time))
        cleaned_out = np.nan_to_num(out)
        return cleaned_out

    def pred_all_users(self, report_run_time=False):
        start_time = time()
        all_ratings = [
            self.pred_one_user(user_id) for user_id in range(self.n_users)]
        if report_run_time:
            print("Execution time: %f seconds" % (time()-start_time))
        return np.array(all_ratings)

    def top_n_recs(self, user_id, n, verbose = False):
        pred_ratings = self.pred_one_user(user_id, report_run_time = verbose)
        item_index_sorted_by_pred_rating = list(np.argsort(pred_ratings))
        items_rated_by_this_user = self.ratings_mat[user_id].nonzero()[1]
        unrated_items_by_pred_rating = [item for item in item_index_sorted_by_pred_rating
                                        if item not in items_rated_by_this_user]
        return unrated_items_by_pred_rating[-n:]


In [134]:
Top_N = 5
recommender = ItemItemRecommender(neighborhood_size=100)
recommender.fit(ratings_mat)
lucky = random.randint(0, highest_user_id - 1)
print('User Chosen is: %d' % lucky)
result = recommender.top_n_recs(lucky, Top_N, verbose = True)
print(result)

User Chosen is: 6730


/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:26: RuntimeWarning: invalid value encountered in divide


Execution time: 0.903347 seconds
[2840, 2738, 2610, 1885, 2171]


In [140]:
# convert into valid business entity
def printBusiness(result_list, unique_user_id):
    business_list = []
    for index in result:
        business_list.append(unique_users_id[index])
    return business_list

In [141]:
convert_result = printBusiness(result, unique_users_id)
print(convert_result)

['g4NQ8YTArwKlCPQx2_GDVw', 'soaoXgD4VKTPxy4Dz8DvZw', '8OjhIQZAOBCEvMhHCnn2pw', 'D43OWyfzIQjL8feJpYh2SQ', 'oU2AHazFini68OjCmQTTrg']


In [148]:
# save the data_df to csv
df_data.to_csv('cleaned_data.csv')

In [149]:
# print out results for all users
Top_N = 5
'''
all_result = [recommender.top_n_recs(index, Top_N) for index in xrange(user_shape[0])]
convert_all_result = [printBusiness(result_list, unique_users_id) for result_list in all_result]
print(convert_all_result)
'''

'\nall_result = [recommender.top_n_recs(index, Top_N) for index in xrange(user_shape[0])]\nconvert_all_result = [printBusiness(result_list, unique_users_id) for result_list in all_result]\nprint(convert_all_result)\n'

## 3. Matrix Factorization recommender

Take a look at Graphlab Create examples

In [56]:
# Using GraphLab Create APIs
import graphlab as gl
sf = gl.SFrame.read_csv('cleaned_data.csv')

Finished parsing file /Users/Peter/Desktop/5000/Data Science/Projects/Python/Yelp/cleaned_data.csv

Parsing completed. Parsed 100 lines in 0.17552 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[int,str,str,int,int,int]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file /Users/Peter/Desktop/5000/Data Science/Projects/Python/Yelp/cleaned_data.csv

Parsing completed. Parsed 157275 lines in 0.196044 secs.

In [66]:
# choose only related information: business_index, user_index, stars (ratings)
data = sf[['business_index', 'user_index', 'stars']]
(train_set, test_set) = data.random_split(0.8)

kfolds = gl.cross_validation.KFold(train_set, 5)
seed = 42
data.head(2)

business_index,user_index,stars
0,0,1
1,0,4


In [67]:
params = dict(user_id='user_index', 
              item_id='business_index', 
              target='stars',
              random_seed = seed)
factor_search = gl.model_parameter_search.create(kfolds,
                                              gl.factorization_recommender.create,
                                              params, max_models = 100)

[INFO] graphlab.deploy.job: Validating job.
[INFO] graphlab.deploy.map_job: Validation complete. Job: 'Model-Parameter-Search-Jun-28-2017-14-35-2200000' ready for execution
[INFO] graphlab.deploy.map_job: Job: 'Model-Parameter-Search-Jun-28-2017-14-35-2200000' scheduled.
[INFO] graphlab.deploy.job: Validating job.
[INFO] graphlab.deploy.map_job: A job with name 'Model-Parameter-Search-Jun-28-2017-14-35-2200000' already exists. Renaming the job to 'Model-Parameter-Search-Jun-28-2017-14-35-2200000-974ea'.
[INFO] graphlab.deploy.map_job: Validation complete. Job: 'Model-Parameter-Search-Jun-28-2017-14-35-2200000-974ea' ready for execution
[INFO] graphlab.deploy.map_job: Job: 'Model-Parameter-Search-Jun-28-2017-14-35-2200000-974ea' scheduled.
[INFO] graphlab.deploy.job: Validating job.
[INFO] graphlab.deploy.map_job: Validation complete. Job: 'Model-Parameter-Search-Jun-28-2017-14-35-2200001' ready for execution
[INFO] graphlab.deploy.map_job: Job: 'Model-Parameter-Search-Jun-28-2017-14-35

In [68]:
from pprint import pprint
factor_search.get_results()

item_id,linear_regularization,max_iterations,num_factors,random_seed,regularization,target
business_index,1e-09,25,8,42,1e-07,stars
business_index,1e-09,50,64,42,1e-06,stars
business_index,1e-07,50,8,42,1e-07,stars
business_index,1e-09,25,32,42,1e-08,stars
business_index,1e-05,50,8,42,1e-08,stars
business_index,1e-09,25,64,42,1e-09,stars
business_index,1e-05,50,8,42,1e-07,stars
business_index,1e-07,25,64,42,1e-08,stars
business_index,1e-05,25,16,42,0.0001,stars
business_index,1e-09,25,64,42,1e-08,stars


In [69]:
factor_best_parameter = factor_search.get_best_params('mean_validation_rmse')
factor_best_parameter

{'item_id': 'business_index',
 'linear_regularization': 1e-09,
 'max_iterations': 50,
 'num_factors': 8,
 'random_seed': 42,
 'regularization': 1e-08,
 'target': 'stars',
 'user_id': 'user_index'}

In [70]:
best_factor_model = gl.factorization_recommender.create(test_set, user_id = 'user_index', item_id = 'business_index',
                                                        target = 'stars', regularization = 10**(-9), num_factors = 8, linear_regularization = 10**(-9))

Recsys training: model = factorization_recommender

Preparing data set.

Data has 31344 observations with 13924 users and 3377 items.

Data prepared in: 0.09205s

Training factorization_recommender for recommendations.

...

+---------+--------------+-------------------+-----------------------+-------------+

Optimization Complete: Maximum number of passes through the data reached.

Computing final objective value and training RMSE.

Final objective value: 0.249954

Final training RMSE: 0.499939

## 4. Other recommenders (optional)

What are other ways you can build a better recommender?

* Other features (have you noticed there are other features in the Yelp dataset, e.g. tips, etc.?)
* Popularity-based
* Content-based
* Hybrid

In [193]:
df.head(5)

,business_id,name,categories,avg_stars,cool,date,funny,review_id,stars,text,type,useful,user_id,count
0,--9e1ONYQuAa-CB_Rrw7Tw,Delmonico Steakhouse,"[Steakhouses, Restaurants, Cajun/Creole]",4.0,0,2015-06-26,0,nCqdz-NW64KazpxqnDr0sQ,1,I mainly went for the ceasar salad prepared ta...,review,0,0XVzm4kVIAaH4eQAxWbhvw,318
1,--9e1ONYQuAa-CB_Rrw7Tw,Delmonico Steakhouse,"[Steakhouses, Restaurants, Cajun/Creole]",4.0,0,2015-06-29,0,iwx6s6yQxc7yjS7NFANZig,4,Nice atmosphere and wonderful service. I had t...,review,0,2aeNFntqY2QDZLADNo8iQQ,318
2,--9e1ONYQuAa-CB_Rrw7Tw,Delmonico Steakhouse,"[Steakhouses, Restaurants, Cajun/Creole]",4.0,0,2015-04-05,0,2HrBENXZTiitcCJfzkELgA,2,To be honest it really quit aweful. First the ...,review,0,WFhv5pMJRDPWSyLnKiWFXA,318
3,--9e1ONYQuAa-CB_Rrw7Tw,Delmonico Steakhouse,"[Steakhouses, Restaurants, Cajun/Creole]",4.0,0,2016-02-16,0,6YNPXoq41qTMZ2TEi0BYUA,2,"The food was decent, but the service was defin...",review,0,2S6gWE-K3DHNcKYYSgN7xA,318
4,--9e1ONYQuAa-CB_Rrw7Tw,Delmonico Steakhouse,"[Steakhouses, Restaurants, Cajun/Creole]",4.0,0,2016-02-08,1,4bQrVUiRZ642odcKCS0OhQ,2,If you're looking for craptastic service and m...,review,1,rCTVWx_Tws2jWi-K89iEyw,318


### When it comes to consider popularity, it will usually related to 'stars'.

In [203]:
# Popularity-based
popularity_model = gl.recommender.popularity_recommender.create(train_set, user_id='user_index',
                                                                item_id='business_index', target = 'stars')

Recsys training: model = popularity

Warning: Ignoring columns X1, business_id, user_id;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 125853 observations with 18893 users and 4065 items.

Data prepared in: 0.155502s

125853 observations to process; with 4065 unique items.

In [204]:
rmse_train = popularity_model['training_rmse']
rmse_test = gl.evaluation.rmse(test_set['stars'], popularity_model.predict(test_set))
print('training set rmse: %f' % rmse_train)
print('test set rmse: %f' % rmse_test)

training set rmse: 1.152739
test set rmse: 1.199568


### When it comes to content based recommender, I think categories is more likely to be considered. Therefore, in this recommender, categories and business_id is considered. 

In [87]:
# Content-based recommender
data = df[['business_id', 'categories']].drop_duplicates()
print(data.shape)
content_sf = gl.SFrame(data = data)

(4358, 2)


In [88]:
content_sf.head(2)

business_id,categories
--9e1ONYQuAa-CB_Rrw7Tw,"[Steakhouses,Restaurants, ..."
-1vfRrlnNnNJ5boOVghMPA,"[Restaurants, Korean,Sushi Bars] ..."


In [89]:
content_sf.shape

(4358, 2)

In [90]:
categories = content_sf['categories']
content_sf.remove_column('categories')
content_sf.head(2)

business_id
--9e1ONYQuAa-CB_Rrw7Tw
-1vfRrlnNnNJ5boOVghMPA


In [91]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
vectorizer = TfidfVectorizer(stop_words='english', max_features=2000)
vectors = vectorizer.fit_transform(categories).toarray()
words = vectorizer.get_feature_names()

In [92]:
vectors.shape

(4358, 419)

In [93]:
category_NLP = gl.SArray(vectors)
content_sf = content_sf.add_column(category_NLP, 'category_NLP')

In [94]:
content_sf.head()

business_id,category_NLP
--9e1ONYQuAa-CB_Rrw7Tw,"[0.0, 0.0, 0.0, 0.0, 0.0,0.0, 0.0, 0.0, 0.0, 0.0, ..."
-1vfRrlnNnNJ5boOVghMPA,"[0.0, 0.0, 0.0, 0.0, 0.0,0.0, 0.0, 0.0, 0.0, 0.0, ..."
-3zffZUHoY8bQjGfPSoBKQ,"[0.0, 0.0, 0.0, 0.0, 0.0,0.0, 0.0, 0.0, 0.0, 0.0, ..."
-8R_-EkGpUhBk55K9Dd4mg,"[0.0, 0.0, 0.0, 0.0, 0.0,0.0, 0.0, 0.0, 0.0, 0.0, ..."
-9YyInW1wapzdNZrhQJ9dg,"[0.0, 0.0, 0.0, 0.0, 0.0,0.0, 0.0, 0.0, 0.0, 0.0, ..."
-AD5PiuJHgdUcAK-Vxao2A,"[0.0, 0.0, 0.0, 0.0, 0.0,0.0, 0.0, 0.0, 0.0, 0.0, ..."
-AGdGGCeTS-njB_8GkUmjQ,"[0.0, 0.0, 0.0, 0.0, 0.0,0.0, 0.0, 0.0, 0.0, 0.0, ..."
-BS4aZAQm9u41YnB9MUASA,"[0.0, 0.0, 0.0, 0.0, 0.0,0.0, 0.0, 0.0, 0.0, 0.0, ..."
-Bf8BQ3yMk8U2f45r2DRKw,"[0.0, 0.0, 0.0, 0.0, 0.0,0.0, 0.0, 0.0, 0.0, 0.0, ..."
-BmqghX1sv7sgsxOIS2yAg,"[0.0, 0.0, 0.0, 0.0, 0.0,0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [95]:
# create recommender
category_content_based = gl.recommender.item_content_recommender.create(content_sf, "business_id")

('Applying transform:\n', Class             : AutoVectorizer

Model Fields
------------
Features          : ['category_NLP']
Excluded Features : ['business_id']

Column        Type   Interpretation  Transforms  Output Type
------------  -----  --------------  ----------  -----------
category_NLP  array  vector          None        array      

)


Recsys training: model = item_content_recommender

Starting brute force nearest neighbors model training.

Starting blockwise querying.

max rows per data block: 7775

...

Data has 0 observations with 0 users and 4358 items.

Data prepared in: 0.269214s

Loading user-provided nearest items.

Generating candidate set for working with new users.

Finished training in 0.017284s

### When it comes to item-item similarity recommender, the business entity is considered as item and users are considered as user while stars are considered as ratings

In [381]:
# item-item similarity recommender
item_item_sf = gl.SFrame(df[['business_id', 'user_id', 'stars']])
item_item_sf = item_item_sf[['user_id', 'business_id', 'stars']]
item_item_sf.head(2)

user_id,business_id,stars
0XVzm4kVIAaH4eQAxWbhvw,--9e1ONYQuAa-CB_Rrw7Tw,1
2aeNFntqY2QDZLADNo8iQQ,--9e1ONYQuAa-CB_Rrw7Tw,4


In [382]:
(train_set, test_set) = item_item_sf.random_split(0.8)

In [387]:
# grid search to find the best parameter set
thresholds = [10**(-i) for i in range(3,10)]
top_k = [2**i for i in range(4, 10)]
similarity_type = ['jaccard', 'cosine', 'pearson']
rmse_train = []
rmse_test = []
parameters = []
for threshold in thresholds:
    for k in top_k:
        for similarity in similarity_type:
            model = gl.item_similarity_recommender.create(train_set,
                                                          user_id = 'user_id', 
                                                          item_id = 'business_id',
                                                          target = 'stars',
                                                          similarity_type = similarity,
                                                          threshold = threshold,
                                                          only_top_k = k,
                                                          verbose = False
                                                         )
            training_rmse = gl.evaluation.rmse(train_set['stars'], model.predict(train_set))
            testing_rmse = gl.evaluation.rmse(test_set['stars'], model.predict(test_set))
            rmse_train.append(training_rmse)
            rmse_test.append(testing_rmse)
            parameters.append((threshold, k, similarity))
            print('rmse_train:%f, rmse_test:%f, threshold:%f, k:%f, similarity:%s' % (training_rmse, testing_rmse, threshold, k, similarity))

Recsys training: model = item_similarity

rmse_train:4.063914, rmse_test:4.062520, threshold:0.001000, k:16.000000, similarity:jaccard


Recsys training: model = item_similarity

rmse_train:4.054978, rmse_test:4.059265, threshold:0.001000, k:16.000000, similarity:cosine


Recsys training: model = item_similarity

...

rmse_train:4.061506, rmse_test:4.060360, threshold:0.000000, k:512.000000, similarity:jaccard


Recsys training: model = item_similarity

rmse_train:4.031261, rmse_test:4.044423, threshold:0.000000, k:512.000000, similarity:cosine


Recsys training: model = item_similarity

rmse_train:1.253611, rmse_test:1.286769, threshold:0.000000, k:512.000000, similarity:pearson


In [388]:
best_index = np.argmin(rmse_test)
best_parameter = parameters[best_index]
print(best_parameter)

(1e-09, 512, 'pearson')


In [390]:
best_item_item_model = gl.item_similarity_recommender.create(train_set,
                                                          user_id = 'user_id', 
                                                          item_id = 'business_id',
                                                          target = 'stars',
                                                          similarity_type = 'pearson',
                                                          threshold = 10**(-9),
                                                          only_top_k = 512,
                                                          verbose = True
                                                         )

Recsys training: model = item_similarity

Preparing data set.

Data has 278458 observations with 138725 users and 4311 items.

Data prepared in: 0.381983s

Training model from provided data.

...

| 262.962ms                           | 100              | 4311            |

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.306563s

### For the Hybrid recommender, I have considered content-based recommender and collaborative filtering recommender to be the best choice

In [86]:
# Let's say I am going to 
sf_all = gl.SFrame(df[['business_id', 'user_id', 'categories', 'stars']])
sf_all.head(2)

business_id,user_id,categories,stars
--9e1ONYQuAa-CB_Rrw7Tw,0XVzm4kVIAaH4eQAxWbhvw,"[Steakhouses,Restaurants, ...",1
--9e1ONYQuAa-CB_Rrw7Tw,2aeNFntqY2QDZLADNo8iQQ,"[Steakhouses,Restaurants, ...",4


Train a content-based recommender

In [96]:
# set up a category map for all business entities
data = df[['business_id', 'categories']].drop_duplicates()
print(data.shape)
content_sf = gl.SFrame(data = data)

(4358, 2)


In [97]:
categories = content_sf['categories']
content_sf.remove_column('categories')
content_sf.head(2)

business_id
--9e1ONYQuAa-CB_Rrw7Tw
-1vfRrlnNnNJ5boOVghMPA


In [98]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
vectorizer = TfidfVectorizer(stop_words='english', max_features=2000)
vectors = vectorizer.fit_transform(categories).toarray()
words = vectorizer.get_feature_names()

In [99]:
category_NLP = gl.SArray(vectors)
content_sf = content_sf.add_column(category_NLP, 'category_NLP')
category_content_based = gl.recommender.item_content_recommender.create(content_sf, "business_id")

('Applying transform:\n', Class             : AutoVectorizer

Model Fields
------------
Features          : ['category_NLP']
Excluded Features : ['business_id']

Column        Type   Interpretation  Transforms  Output Type
------------  -----  --------------  ----------  -----------
category_NLP  array  vector          None        array      

)


Recsys training: model = item_content_recommender

Starting brute force nearest neighbors model training.

Starting blockwise querying.

max rows per data block: 7775

...

Data has 0 observations with 0 users and 4358 items.

Data prepared in: 0.317056s

Loading user-provided nearest items.

Generating candidate set for working with new users.

Finished training in 0.013906s

### Cascade Hybrid Recommender System
This hybrid recommender system is to make use of both content based and collaborative filtering recommendation system to only evaluate the business entities that are very close to chosen category and predict the ratings based on only these entities.

In [104]:
import random
shape = sf_all.shape
rand_index = random.randint(0, shape[0])
chosen_row = sf_all[rand_index]
chosen_row

{'business_id': 'iTCH7y2KLRMKMxdEkgUK1g',
 'categories': '[Restaurants, Food Trucks, Venezuelan, Food, Latin American]',
 'stars': 5,
 'user_id': 'e3c6w6sM8sJ_g0IlF6CSog'}

In [108]:
# choose 300 closest business entities
choose_k = 300
chosen_k_entities = category_content_based.recommend_from_interactions([chosen_row['business_id']], k = choose_k)
chosen_k_entities.head(2)

business_id,score,rank
1uDUisf3ro5V3vC60cxfFw,1.0,1
EnCIojgP5KTr1leaysFE3A,0.763970792294,2


In [116]:
df_chosen_business_id = chosen_k_entities[['business_id']].to_dataframe()
df_filtered = pd.merge(df, df_chosen_business_id, left_on = 'business_id', right_on = 'business_id', how = 'inner')
df_filtered.head(2)

,business_id,name,categories,avg_stars,cool,date,funny,review_id,stars,text,type,useful,user_id,count
0,-mN7z9oY01Mh_-dwTyzpqg,Einstein Bros Bagels,"[Breakfast & Brunch, Restaurants]",3.5,1,2016-12-10,0,bwTUmpkPpZKbtVuHl-vE9Q,4,I think that this Einstein's is one of the bet...,review,0,E63DvbIQptvI05F63F1Bdg,36
1,-mN7z9oY01Mh_-dwTyzpqg,Einstein Bros Bagels,"[Breakfast & Brunch, Restaurants]",3.5,1,2015-06-27,0,5occn6g-CNryBX1Eojgp7A,5,I have been to Einstein locations all over Veg...,review,0,lqKZOuHeqoY3WdwBQj7gZw,36


In [117]:
cf_sf = gl.SFrame(df_filtered[['business_id', 'user_id', 'stars']])
cf_sf.head(2)

business_id,user_id,stars
-mN7z9oY01Mh_-dwTyzpqg,E63DvbIQptvI05F63F1Bdg,4
-mN7z9oY01Mh_-dwTyzpqg,lqKZOuHeqoY3WdwBQj7gZw,5


In [118]:
cf_sf.shape

(22040, 3)

In [119]:
(train_set, test_set) = cf_sf.random_split(0.7)

In [124]:
CF_model = gl.item_similarity_recommender.create(train_set,
                                                          user_id = 'user_id', 
                                                          item_id = 'business_id',
                                                          target = 'stars',
                                                          similarity_type = 'pearson',
                                                          threshold = 10**(-9),
                                                          only_top_k = 512,
                                                          verbose = True
                                                         )

Recsys training: model = item_similarity

Preparing data set.

Data has 15351 observations with 13326 users and 296 items.

Data prepared in: 0.040412s

Training model from provided data.

...

| 27.819ms                            | 100              | 296             |

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.037583s

In [125]:
score = gl.evaluation.rmse(test_set['stars'], CF_model.predict(test_set))
score

1.2495618522895826

In [127]:
recommend_result = CF_model.recommend(new_observation_data=test_set)

recommendations finished on 1000/13326 queries. users per second: 88896.8

recommendations finished on 2000/13326 queries. users per second: 89710.2

recommendations finished on 3000/13326 queries. users per second: 90435

recommendations finished on 4000/13326 queries. users per second: 92504.8

recommendations finished on 5000/13326 queries. users per second: 94158.4

...

recommendations finished on 9000/13326 queries. users per second: 88247.4

recommendations finished on 10000/13326 queries. users per second: 87849.5

recommendations finished on 11000/13326 queries. users per second: 88340.6

recommendations finished on 12000/13326 queries. users per second: 88140

recommendations finished on 13000/13326 queries. users per second: 87423.8

In [128]:
recommend_result

user_id,business_id,score,rank
E63DvbIQptvI05F63F1Bdg,EUskCVPgHoIGl3Ao0VUNLA,5.0,1
E63DvbIQptvI05F63F1Bdg,8hZjiPzJIojA1k7_W4hELA,5.0,2
E63DvbIQptvI05F63F1Bdg,3nWXxSCR6q7SVxHvsUUeWg,5.0,3
E63DvbIQptvI05F63F1Bdg,3PshdJtSwd_poaPL7fIOHg,5.0,4
E63DvbIQptvI05F63F1Bdg,39BNnMLzx0UUwkWKxGeQKA,5.0,5
E63DvbIQptvI05F63F1Bdg,35tWXOOJpWB2feUAEJyJyg,5.0,6
E63DvbIQptvI05F63F1Bdg,2sjfncn3GNHpKZDAcsm1Uw,5.0,7
E63DvbIQptvI05F63F1Bdg,0i9S0BejjRv0ZDwdO9XymA,5.0,8
E63DvbIQptvI05F63F1Bdg,0N53m33GANYeHHl-s22d4Q,5.0,9
E63DvbIQptvI05F63F1Bdg,-xbQQR_ydEJGqYzHSF4DnQ,5.0,10


### Switching.
When a (relatively) new user comes, very few ratings has been made, then content-based recommender is used. 
This might on some level solve the 'cold start' problem

In [165]:
category_content_based = gl.recommender.item_content_recommender.create(content_sf, "business_id")

('Applying transform:\n', Class             : AutoVectorizer

Model Fields
------------
Features          : ['category_NLP']
Excluded Features : ['business_id']

Column        Type   Interpretation  Transforms  Output Type
------------  -----  --------------  ----------  -----------
category_NLP  array  vector          None        array      

)


Recsys training: model = item_content_recommender

Starting brute force nearest neighbors model training.

Starting blockwise querying.

max rows per data block: 7775

...

Data has 0 observations with 0 users and 4358 items.

Data prepared in: 0.311917s

Loading user-provided nearest items.

Generating candidate set for working with new users.

Finished training in 0.018105s

In [169]:
shape = df.shape
random_row = random.randint(0, shape[0])
chosen_user = df.iloc[random_row,].user_id
df_chosen_user = df[df['user_id'] == chosen_user]
print('chosen user is: %s' % chosen_user)
df_chosen_user

chosen user is: sa4b69yUwJ6QeHV20XPnNg


,business_id,name,categories,avg_stars,cool,date,funny,review_id,stars,text,type,useful,user_id,count
309713,rbDqCV2g23K3ZrTxmgoNBg,"Biscayne Steak, Sea and Wine","[American (New), Steakhouses, Restaurants, Sea...",4.0,0,2016-11-05,1,c5WPPFXpvB2aL1dUTZ9rcw,4,I had the bone in filet and my partner had the...,review,1,sa4b69yUwJ6QeHV20XPnNg,49


In [170]:
high_rate_business_for_user = list(df_chosen_user[df_chosen_user['stars'] >= 3]['business_id'])
high_rate_business_for_user

['rbDqCV2g23K3ZrTxmgoNBg']

In [173]:
if (len(high_rate_business_for_user) > 0):
    recommend_result = None
    threshold = 2
    if df_chosen_user.shape[0] >= 10:
        # use item-similarity recommender
        CF_model = gl.item_similarity_recommender.create(sf_all,
                                                         user_id = 'user_id',
                                                         item_id = 'business_id',
                                                         target = 'stars',
                                                         similarity_type = 'pearson',
                                                         threshold = 10**(-9),
                                                         only_top_k = 512,
                                                         verbose = True
                                                        )
        recommend_result = CF_model.recomend(k = 5)
    else:
        # use content-based recommender
        recommend_result = category_content_based.recommend_from_interactions(high_rate_business_for_user, k = 5)
    recommend_df = recommend_result.to_dataframe()
    all_information = pd.merge(df[['business_id', 'avg_stars', 'name', 'categories']].drop_duplicates(), recommend_df,
                               how='inner', left_on = 'business_id', right_on= 'business_id')
    print(all_information.head())
else:
    print('No quality rating for this user')

              business_id  avg_stars                        name  \
0  3Gt3xskppi9jZuTrwrhLNg        4.0      Stack Restaurant & Bar   
1  7UFDAX4wLi6ux5otguYldA        4.0         Andiron Steak & Sea   
2  bjSC_jbrypke0l-bXXBmwQ        4.5  Vic & Anthony's Steakhouse   
3  jCR-xC4NqoEajjmstqX8sA        4.0                 Twin Creeks   
4  tQifTiY-vutj8orxcMJKfQ        4.0         Triple George Grill   

                                          categories     score  rank  
0  [Seafood, Restaurants, American (New), Steakho...  1.000000     4  
1  [Steakhouses, Seafood, American (New), Restaur...  1.000000     3  
2  [American (Traditional), Restaurants, Steakhou...  0.903751     5  
3  [Restaurants, Seafood, Steakhouses, American (...  1.000000     2  
4  [Restaurants, Steakhouses, Seafood, American (...  1.000000     1  
